In [1]:
import os

In [2]:
# import dagshub
# dagshub.init(repo_owner='mann-lean', repo_name='data-science-project', mlflow=True)

# import mlflow
# with mlflow.start_run():
#   mlflow.log_param('parameter name', 'value')
#   mlflow.log_metric('metric name', 1)

In [3]:
# set environment variables for MLFLOW
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/mann-lean/data-science-project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="mann-lean"
os.environ["MLFLOW_TRACKING_PASSWORD"]="d48e826348084596921be95179ff847d63506cba"

In [4]:
%pwd

'e:\\dsProject\\nycTaxiProject\\research'

In [5]:
os.chdir('../')

In [6]:
%pwd

'e:\\dsProject\\nycTaxiProject'

In [7]:
from dataclasses import dataclass
from pathlib import Path

In [8]:
# Entity
@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    x_train_dir: Path
    y_train_dir: Path
    x_test_dir: Path
    y_test_dir: Path
    model_dir: Path

In [9]:
from nycTaxiProject.util.common import read_yaml,create_directories
from nycTaxiProject.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH

In [10]:
class ConfigurationManger:
    def __init__(
            self,
     config_file_path=CONFIG_FILE_PATH,
     params_file_path=PARAMS_FILE_PATH
     ):
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)

    def get_model_evaluation(self)->ModelEvaluationConfig:
        config=self.config.model_evaluation
        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            x_train_dir=config.x_train_dir,
            y_train_dir=config.y_train_dir,
            x_test_dir=config.x_test_dir,
            y_test_dir=config.y_test_dir,
            model_dir=config.model_dir
        )
        return model_evaluation_config

In [11]:
import logging
logger=logging.getLogger(__name__)
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.linear_model import SGDRegressor
import numpy as np
import joblib
import pandas as pd

In [12]:
class Model_Evaluation:
    def __init__(self,config=ModelEvaluationConfig):
        self.config=config
    def evaluate_model(self, X, y_true, name:str): #why argument  need model,when we are loading the model from the directory? because we are loading the model from the directory, we don't need to pass the model as an argument to the evaluate_model method. We can remove the model argument from the method definition and directly load the model within the method using joblib.load(self.config.model_dir). Here's how you can modify the evaluate_model method:
        try:
            model=joblib.load(self.config.model_dir)
            y_pred = model.predict(X)
            r2 = r2_score(y_true, y_pred)
            rmse = np.sqrt(mean_squared_error(y_true, y_pred))
            mse=mean_squared_error(y_true, y_pred)
            mae = mean_absolute_error(y_true, y_pred)
            
            print(f"-----------{name} Evaluation-------------")
            print(f"R2 Score:   {r2:.4f}")
            print(f"RMSE:       {rmse:.4f}")
            print(f"MSE:        {mse:.4f}")
            print(f"MAE:        {mae:.4f}")
        except Exception as e:
            logger.exception(e)
            raise e


In [17]:
try:
    logger.info(">>>>>>>STAGE: MODEL EVALUATION Started<<<<<<<<< ")
    config=ConfigurationManger()
    model_evaluation_config=config.get_model_evaluation()
    x_train=pd.read_csv(model_evaluation_config.x_train_dir)
    y_train=pd.read_csv(model_evaluation_config.y_train_dir)
    model_evaluation=Model_Evaluation(model_evaluation_config)
    model_evaluation.evaluate_model(x_train,y_train,"training")

    x_test=pd.read_csv(model_evaluation_config.x_test_dir)
    y_test=pd.read_csv(model_evaluation_config.y_test_dir)
    model_evaluation.evaluate_model(x_test,y_test,"testing")
    logger.info(">>>>>>>STAGE: MODEL EVALUATION ENDED<<<<<<<<< ")

except Exception as e:
    logger.exception(e)
    raise e

[2026-02-25 02:17:28,549 : INFO : 2382430873 : >>>>>>>STAGE: MODEL EVALUATION Started<<<<<<<<< ]
[2026-02-25 02:17:28,571 : INFO : common : yaml file: config\config.yaml LOADED successfully]
[2026-02-25 02:17:28,585 : INFO : common : yaml file: params.yaml LOADED successfully]
[2026-02-25 02:17:28,591 : INFO : common : created directory at: artifacts/modelEvaluatioin]
-----------training Evaluation-------------
R2 Score:   0.6906
RMSE:       2.2370
MSE:        5.0043
MAE:        1.1405
-----------testing Evaluation-------------
R2 Score:   0.7187
RMSE:       2.0790
MSE:        4.3223
MAE:        1.1287
[2026-02-25 02:17:36,645 : INFO : 2382430873 : >>>>>>>STAGE: MODEL EVALUATION ENDED<<<<<<<<< ]
